In [1]:
from newspaper import Article
import pandas as pd
import re
import torch

In [2]:
def clean(x):
    return x.replace("(","").replace(")","").replace("{","").replace("}","").replace("[","").replace("]","").replace('"',"").replace("'","").replace("=","")
def _get_ngrams(n, text):
    """Calcualtes n-grams.

    Args:
      n: which n-grams to calculate
      text: An array of tokens

    Returns:
      A set of n-grams
    """
    ngram_set = set()
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.add(tuple(text[i:i + n]))
    return ngram_set


def _get_word_ngrams(n, sentences):
    """Calculates word n-grams for multiple sentences.
    """
    assert len(sentences) > 0
    assert n > 0

    # words = _split_into_words(sentences)

    words = sum(sentences, [])
    # words = [w for w in words if w not in stopwords]
    return _get_ngrams(n, words)

In [10]:
from konlpy.tag import Kkma
def load_data(row):
    kkma=Kkma()
    article=row["body"]
    abstract=row["tgt"]
#     article=article.text
#     abstract=tgt_text
    src_txt=kkma.sentences(article)
    tgt_txt=kkma.sentences(abstract)

    source=[kkma.morphs(clean(sen)) for sen in src_txt]
    tgt=[kkma.morphs(clean(sen)) for sen in tgt_txt]
    return source,tgt
def _format_to_lines(f):
    source, tgt = load_data(f)
    return {'src': source, 'tgt': tgt}

def format_to_lines():
    for corpus_type in ['valid', 'test', 'train']:
        df=pd.read_csv(corpus_type+"_data.csv")
        df.apply(_format_to_lines,axis=1).to_json("cnndm_sample."+corpus_type+".0"+".json",orient="records")
format_to_lines()